In [1]:
# Familiar imports
import numpy as np
import pandas as pd
import random
import os
import time
from pathlib import Path

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder

import lightgbm as lgb
#import xgboost as xgb
#import catboost as ctb

import warnings
warnings.simplefilter('ignore')

# Parameters

In [2]:
target = 'target'

DEBUG = False

if DEBUG:
    N_ESTIMATORS = 1
    N_SPLITS = 2
    SEED = 17
    CVSEED = 17
    EARLY_STOPPING_ROUNDS = 1
    VERBOSE = 100
    #N_ITERS = 2
else:
    N_SPLITS = 10
    N_ESTIMATORS = 20000
    EARLY_STOPPING_ROUNDS = 200
    VERBOSE = 1000
    SEED = 17
    CVSEED = 17
    #N_ITERS = 10

In [3]:
def set_seed(seed=17):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    
set_seed(SEED)

# Load data

In [4]:
# Load the training data
train = pd.read_csv("../input/30-days-of-ml/train.csv")
test = pd.read_csv("../input/30-days-of-ml/test.csv")

# Preview the data
train.head()

,id,cat0,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,...,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13,target
0,1,B,B,B,C,B,B,A,E,C,...,0.400361,0.160266,0.310921,0.389470,0.267559,0.237281,0.377873,0.322401,0.869850,8.113634
1,2,B,B,A,A,B,D,A,F,A,...,0.533087,0.558922,0.516294,0.594928,0.341439,0.906013,0.921701,0.261975,0.465083,8.481233
2,3,A,A,A,C,B,D,A,D,A,...,0.650609,0.375348,0.902567,0.555205,0.843531,0.748809,0.620126,0.541474,0.763846,8.364351
3,4,B,B,A,C,B,D,A,E,C,...,0.668980,0.239061,0.732948,0.679618,0.574844,0.346010,0.714610,0.540150,0.280682,8.049253
4,6,A,A,A,C,B,D,A,E,A,...,0.686964,0.420667,0.648182,0.684501,0.956692,1.000773,0.776742,0.625849,0.250823,7.972260


In [5]:
# Separate target from features
y = train['target']
features = train.drop(['id','target'], axis=1)

# Preview features
features.head()

,cat0,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,...,cont4,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13
0,B,B,B,C,B,B,A,E,C,N,...,0.610706,0.400361,0.160266,0.310921,0.389470,0.267559,0.237281,0.377873,0.322401,0.869850
1,B,B,A,A,B,D,A,F,A,O,...,0.276853,0.533087,0.558922,0.516294,0.594928,0.341439,0.906013,0.921701,0.261975,0.465083
2,A,A,A,C,B,D,A,D,A,F,...,0.285074,0.650609,0.375348,0.902567,0.555205,0.843531,0.748809,0.620126,0.541474,0.763846
3,B,B,A,C,B,D,A,E,C,K,...,0.284667,0.668980,0.239061,0.732948,0.679618,0.574844,0.346010,0.714610,0.540150,0.280682
4,A,A,A,C,B,D,A,E,A,N,...,0.287595,0.686964,0.420667,0.648182,0.684501,0.956692,1.000773,0.776742,0.625849,0.250823


# Preprocessing

In [6]:
'''
# List of categorical columns
object_cols = [col for col in features.columns if 'cat' in col]

# ordinal-encode categorical columns
X = features.copy()
X_test = test.drop(['id'], axis=1).copy()
ordinal_encoder = OrdinalEncoder()
X[object_cols] = ordinal_encoder.fit_transform(features[object_cols])
X_test[object_cols] = ordinal_encoder.transform(test[object_cols])

# Preview the ordinal-encoded features
X.head()
'''

"\n# List of categorical columns\nobject_cols = [col for col in features.columns if 'cat' in col]\n\n# ordinal-encode categorical columns\nX = features.copy()\nX_test = test.drop(['id'], axis=1).copy()\nordinal_encoder = OrdinalEncoder()\nX[object_cols] = ordinal_encoder.fit_transform(features[object_cols])\nX_test[object_cols] = ordinal_encoder.transform(test[object_cols])\n\n# Preview the ordinal-encoded features\nX.head()\n"

In [7]:

# List of categorical columns
object_cols = [col for col in features.columns if 'cat' in col]

# one-hot-encode categorical columns
X = features.copy()
X_test = test.drop(['id'], axis=1).copy()

oh_encoder = OneHotEncoder(sparse=False, handle_unknown="ignore")
X_ohe = oh_encoder.fit_transform(features[object_cols])
X_test_ohe = oh_encoder.transform(test[object_cols])

X_ohe = pd.DataFrame(X_ohe, columns=[f"ohe_{i}" for i in range(X_ohe.shape[1])])
X_test_ohe = pd.DataFrame(X_test_ohe, columns=[f"ohe_{i}" for i in range(X_test_ohe.shape[1])])

X = pd.concat([X, X_ohe], axis=1)
X_test = pd.concat([X_test, X_test_ohe], axis=1)
X = X.drop(object_cols, axis=1)
X_test = X_test.drop(object_cols, axis=1)
    
# Preview the one-hot-encoded features
X.head()


,cont0,cont1,cont2,cont3,cont4,cont5,cont6,cont7,cont8,cont9,...,ohe_46,ohe_47,ohe_48,ohe_49,ohe_50,ohe_51,ohe_52,ohe_53,ohe_54,ohe_55
0,0.201470,-0.014822,0.669699,0.136278,0.610706,0.400361,0.160266,0.310921,0.389470,0.267559,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.743068,0.367411,1.021605,0.365798,0.276853,0.533087,0.558922,0.516294,0.594928,0.341439,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,0.742708,0.310383,-0.012673,0.576957,0.285074,0.650609,0.375348,0.902567,0.555205,0.843531,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.429551,0.620998,0.577942,0.280610,0.284667,0.668980,0.239061,0.732948,0.679618,0.574844,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,1.058291,0.367492,-0.052389,0.232407,0.287595,0.686964,0.420667,0.648182,0.684501,0.956692,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [8]:
scale_features = [col for col in features.columns if 'cont' in col]

ss = StandardScaler()
X[scale_features] = ss.fit_transform(features[scale_features])
X_test[scale_features] = ss.transform(test[scale_features])

In [9]:
# Swap noise

# Random
def apply_noise_rn(df, p=.75):
    should_not_swap = np.random.binomial(1, p, df.shape)
    corrupted_df = df.where(should_not_swap == 1, np.random.permutation(df))
    return corrupted_df

# Row-wise
def apply_noise_row(df, p=.75):
    should_not_swap = np.zeros(df.shape)
    for i in range(df.shape[0]):
        for j in np.random.choice(df.shape[1],int(p*df.shape[1]),replace=False):
            should_not_swap[i,j]=1 
    corrupted_df = df.where(should_not_swap == 1, np.random.permutation(df))
    return corrupted_df

# Optuna

In [10]:
# Optuna for parameter search
!pip install -q optuna

import optuna
import pickle

In [11]:
# for the fixed learning rate, use the opt n iterations and tune the tree hyperparameters
def objective(trial, X=X, y=y):
  """
  """
  param_space = {
               #'device':'gpu',  # Use GPU acceleration
               #'boosting_type': 'gbdt',
               'reg_lambda':trial.suggest_uniform('reg_lambda', 28.5, 52.5),
              'reg_alpha':trial.suggest_uniform('reg_alpha', 23.66, 47.66),
                'subsample': trial.suggest_uniform('subsample', 0.79, 1.0),
              'colsample_bytree':trial.suggest_uniform('colsample_bytree', 0.004, 0.404),
                #'subsample_freq': trial.suggest_int('subsample_freq', 1, 10),
               'learning_rate':trial.suggest_uniform('learning_rate', 1e-2, 1e-1),
                 'min_child_samples':trial.suggest_int('min_child_samples', 3, 50),
              'num_leaves':trial.suggest_int('num_leaves', 58, 158),
              'max_depth':trial.suggest_int('max_depth', 2, 10),
              #'min_split_gain': trial.suggest_float('min_split_gain', 0.0, 0.005),
              #'class_weight':trial.suggest_categorical('class_weight',['balanced',None]),
               'n_estimators':N_ESTIMATORS,
                'n_jobs' : -1,
              'metric':'rmse',
              'max_bin':trial.suggest_int('max_bin', 632, 1032),
              'cat_smooth':trial.suggest_int('cat_smooth', 59, 109),
              'cat_l2':trial.suggest_loguniform('cat_l2', 1e-3, 50)
                }
            
  seed_list=[SEED, SEED+1]
  #kf = KFold(n_splits=N_SPLITS, shuffle=True, random_state=CVSEED)
  #lgb_oof = np.zeros(train.shape[0])
  #lgb_pred = np.zeros(test.shape[0])
  X_tr, X_va, y_tr, y_va = train_test_split(X,y,test_size=.2,random_state=CVSEED)
  pred=np.zeros(y_va.shape[0])
    
  #for (trn_idx, val_idx) in enumerate(kf.split(X, y)):
  #print(f"===== fold {fold} =====")
    
  #X_tr = apply_noise_row(X_tr)

  
  #start = time.time()
  for inseed in seed_list:
    param_space['random_state'] = inseed

    model = lgb.LGBMRegressor(**param_space)
    model.fit(
        X_tr, 
        y_tr,
        eval_set=[(X_va, y_va)],
        eval_metric='rmse',
        early_stopping_rounds=EARLY_STOPPING_ROUNDS,
        verbose=VERBOSE,
        #categorical_feature=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
        #callbacks=[optuna.integration.LightGBMPruningCallback(trial, metric='rmse')],
    )


    pred += model.predict(X_va) / len(seed_list)
    
  #elapsed = time.time() - start
  rmse = mean_squared_error(y_va, pred, squared=False)
  #print(f"fold {fold} - lgb rmse: {rmse:.6f}, elapsed time: {elapsed:.2f}sec\n")            
  
  #print(f"oof lgb_rmse = {mean_squared_error(train[target], lgb_oof, squared=False)}")

  
  return rmse

In [12]:
study = optuna.create_study(direction='minimize')
study.optimize(objective,n_trials= 30)
print('Number of finished trials:', len(study.trials))
print('Best trial:', study.best_trial.params)

[I 2021-08-29 14:01:21,451] A new study created in memory with name: no-name-5b606795-88d4-401e-bfbc-017fff26d4e6


Training until validation scores don't improve for 200 rounds
[1000]	valid_0's rmse: 0.71785
[2000]	valid_0's rmse: 0.715354
[3000]	valid_0's rmse: 0.714908
Early stopping, best iteration is:
[3661]	valid_0's rmse: 0.714819
Training until validation scores don't improve for 200 rounds
[1000]	valid_0's rmse: 0.718571
[2000]	valid_0's rmse: 0.715644
[3000]	valid_0's rmse: 0.71512
Early stopping, best iteration is:
[3347]	valid_0's rmse: 0.715081


[I 2021-08-29 14:04:48,528] Trial 0 finished with value: 0.7146375097508282 and parameters: {'reg_lambda': 35.27947375497561, 'reg_alpha': 37.21468181236867, 'subsample': 0.8488162149702033, 'colsample_bytree': 0.21194302543539598, 'learning_rate': 0.014673599405678691, 'min_child_samples': 38, 'num_leaves': 151, 'max_depth': 8, 'max_bin': 697, 'cat_smooth': 62, 'cat_l2': 0.8044858942544637}. Best is trial 0 with value: 0.7146375097508282.


Training until validation scores don't improve for 200 rounds
[1000]	valid_0's rmse: 0.717072
[2000]	valid_0's rmse: 0.714736
[3000]	valid_0's rmse: 0.714371
Early stopping, best iteration is:
[3010]	valid_0's rmse: 0.714369
Training until validation scores don't improve for 200 rounds
[1000]	valid_0's rmse: 0.71988
[2000]	valid_0's rmse: 0.715973
[3000]	valid_0's rmse: 0.715245
Early stopping, best iteration is:
[3479]	valid_0's rmse: 0.715094


[I 2021-08-29 14:06:55,742] Trial 1 finished with value: 0.7142804723015487 and parameters: {'reg_lambda': 42.93588471342488, 'reg_alpha': 32.302778394338375, 'subsample': 0.8875421809598261, 'colsample_bytree': 0.06423307611262974, 'learning_rate': 0.03293684527289206, 'min_child_samples': 21, 'num_leaves': 133, 'max_depth': 9, 'max_bin': 736, 'cat_smooth': 84, 'cat_l2': 0.045864066849847604}. Best is trial 1 with value: 0.7142804723015487.


...

Training until validation scores don't improve for 200 rounds
[1000]	valid_0's rmse: 0.724243
[2000]	valid_0's rmse: 0.719637
[3000]	valid_0's rmse: 0.717549
[4000]	valid_0's rmse: 0.716223
[5000]	valid_0's rmse: 0.715474
[6000]	valid_0's rmse: 0.714935
[7000]	valid_0's rmse: 0.714588
[8000]	valid_0's rmse: 0.714299
[9000]	valid_0's rmse: 0.714157
[10000]	valid_0's rmse: 0.714052
[11000]	valid_0's rmse: 0.713955
[12000]	valid_0's rmse: 0.713901
Early stopping, best iteration is:
[11980]	valid_0's rmse: 0.713896
Training until validation scores don't improve for 200 rounds
[1000]	valid_0's rmse: 0.723948
[2000]	valid_0's rmse: 0.719796
[3000]	valid_0's rmse: 0.717392
[4000]	valid_0's rmse: 0.716206
[5000]	valid_0's rmse: 0.715447
[6000]	valid_0's rmse: 0.714873
[7000]	valid_0's rmse: 0.714541
[8000]	valid_0's rmse: 0.714278
[9000]	valid_0's rmse: 0.7141
[10000]	valid_0's rmse: 0.714008
[11000]	valid_0's rmse: 0.713899
[12000]	valid_0's rmse: 0.713829
Early stopping, best iteration is:
[

[I 2021-08-29 15:22:10,944] Trial 28 finished with value: 0.7137679030543781 and parameters: {'reg_lambda': 42.13735356662759, 'reg_alpha': 36.14968851846132, 'subsample': 0.9159595470755406, 'colsample_bytree': 0.04226565784406397, 'learning_rate': 0.03872902654480494, 'min_child_samples': 40, 'num_leaves': 75, 'max_depth': 3, 'max_bin': 968, 'cat_smooth': 73, 'cat_l2': 0.0010895029069886582}. Best is trial 25 with value: 0.7135805321548033.


Training until validation scores don't improve for 200 rounds
[1000]	valid_0's rmse: 0.717834
[2000]	valid_0's rmse: 0.715348
[3000]	valid_0's rmse: 0.714625
[4000]	valid_0's rmse: 0.714447
Early stopping, best iteration is:
[4100]	valid_0's rmse: 0.714429
Training until validation scores don't improve for 200 rounds
[1000]	valid_0's rmse: 0.717866
[2000]	valid_0's rmse: 0.71544
[3000]	valid_0's rmse: 0.714686
[4000]	valid_0's rmse: 0.714513
Early stopping, best iteration is:
[4166]	valid_0's rmse: 0.714493


[I 2021-08-29 15:25:19,397] Trial 29 finished with value: 0.7143512267938333 and parameters: {'reg_lambda': 45.142900629819074, 'reg_alpha': 37.09410010960552, 'subsample': 0.9595494045949351, 'colsample_bytree': 0.23565381130545318, 'learning_rate': 0.0188244065800554, 'min_child_samples': 43, 'num_leaves': 85, 'max_depth': 6, 'max_bin': 1032, 'cat_smooth': 64, 'cat_l2': 0.009414837433348032}. Best is trial 25 with value: 0.7135805321548033.


Number of finished trials: 30
Best trial: {'reg_lambda': 41.78296284134347, 'reg_alpha': 38.33783791655099, 'subsample': 0.976360319972227, 'colsample_bytree': 0.09092207097566063, 'learning_rate': 0.022034096577430256, 'min_child_samples': 34, 'num_leaves': 67, 'max_depth': 3, 'max_bin': 1029, 'cat_smooth': 70, 'cat_l2': 0.0010825325380121003}


In [13]:
study.best_params

{'reg_lambda': 41.78296284134347,
 'reg_alpha': 38.33783791655099,
 'subsample': 0.976360319972227,
 'colsample_bytree': 0.09092207097566063,
 'learning_rate': 0.022034096577430256,
 'min_child_samples': 34,
 'num_leaves': 67,
 'max_depth': 3,
 'max_bin': 1029,
 'cat_smooth': 70,
 'cat_l2': 0.0010825325380121003}

# Log

====== Ordinal encoding =========

0.7179915711010899 row-wise noise ver4

0.7180313553153085 random noise ver5

0.7145452167048295 no noise ver6

0.7143969338945801 no noise ver10 (narrow space)

0.7141318460599843 no noise ver11 (narrow space)

0.7140261333995335 no noise ver12 (narrow space)

0.7139793127526392 no noise ver13 (narrow space) final

====== One-hot encoding =========

0.7242254028037789 no noise ver7

0.7264762744397595 random noise ver8

0.7265375795882082 row-wise noise ver9

0.7138071888962808 no noise ver16

narrow
